In [1]:
import ROOT
from rootpy.tree import Tree, TreeModel, FloatCol, IntCol
from rootpy.io import root_open
from ROOT import gROOT, TCanvas, TF1, TFile, TTree, gRandom, TH1F

from ROOT import RooRealVar, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooGaussian,RooExponential, \
                 RooBinning, kRed, kBlue, kDotted,TString,RooAbsData, RooPlot, TCut, RooAbsData

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, random

from ROOT import TTree, TFile

# from root_numpy import root2array, rec2array, array2root

import pandas as pd
import numpy as np
import scipy 
import root_pandas as rp
import root_numpy as ry 

import pandas.core.common as com
from pandas.core.index import Index
from pandas.tools import plotting
from pandas.tools.plotting import scatter_matrix

from tqdm import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

sys.path.append('/home/chasenberg/repos/')
sys.path.append('/home/chasenberg/repos/dopy')
from dopy import * 
from dopy.dolearn.sklearn_utils import plot_roc_curve, plot_classifier_output, plot_correlations
from dopy.dolearn.sklearn_utils import plot_feature_importances, plot_classifier_output, classify_unseen_data
#from dopy.sklearn_utils import plot_bdt_vars
from dopy.doplot.plotting import Plotter, Plot
from dopy.doanalysis.df_utils import add_min_max, add_eta 

/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Create new ROOT File with relevant branches

In [3]:
#Names of the produced tuples
mc_dir_2015_2016 = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/'
mc_sanity_cuts = 'Bd2JpsiKS_sanity.root'
mc_bestPV = 'Bd2JpsiKS_bestPV.root'
mc_random = 'Bd2JpsiKS_random.root'
mc_selected = 'Bd2JpsiKS_selected.root'

In [4]:
cut_string = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0&(B0_L0MuonDecision_Dec==1|B0_L0DiMuonDecision_Dec==1|B0_L0MuonHighDecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt2DiMuonJPsiDecision_Dec==1|B0_Hlt2DiMuonDetachedJPsiDecision_Dec==1)'               
tree_mc = 'Bd2JpsiKs'

In [5]:
calculate_trigger_efficiency = True 
create_sanity_tuple = True
#create_bestPV_tuple = True
create_randomSel_tuple = True
create_training_tuple = False
create_l0veto_tuple = True
#do_sanity_massfit = True
#do_bestPV_massfit = True 
#do_randomPV_massfit = True 

In [9]:
variables = [
#Flavour tagging
'B0_TAGOMEGA_OS',
'B0_TAGDECISION_OS',
#Mass and kinematics -> Constrained daughters
'B0_FitDaughtersConst_status_flat',
'B0_FitDaughtersConst_M_flat',
'B0_FitDaughtersConst_chi2_flat',
'B0_FitDaughtersConst_IPCHI2_flat',
'B0_FitDaughtersConst_nDOF_flat',
'B0_FitDaughtersConst_PT_flat',
#Jpsi
'B0_FitDaughtersConst_J_psi_1S_P0_PT_flat', 
'B0_FitDaughtersConst_J_psi_1S_P1_PT_flat',
'B0_FitDaughtersConst_J_psi_1S_MinIPCHI2anyPV_flat',
'B0_FitDaughtersConst_J_psi_1S_IPCHI2_flat',
'B0_FitDaughtersConst_J_psi_1S_IP_flat', 
#KS0
'B0_FitDaughtersConst_KS0_P1_PT_flat', 
'B0_FitDaughtersConst_KS0_P0_PT_flat',
'B0_FitDaughtersConst_KS0_MinIPCHI2anyPV',  
'B0_FitDaughtersConst_KS0_P0_IPCHI2_flat', 
'B0_FitDaughtersConst_KS0_P1_IPCHI2_flat',
'B0_FitDaughtersConst_KS0_decayLength_flat',
'B0_FitDaughtersConst_KS0_IP_flat',

#Mass and kinematics -> Constrained pv
'B0_FitPVConst_status_flat',
'B0_FitPVConst_IPCHI2_flat',
'B0_FitPVConst_chi2_flat',
'B0_FitPVConst_nDOF_flat',
'B0_FitPVConst_tauErr_flat',
'B0_FitPVConst_tau_flat',
'B0_FitPVConst_MinIPCHI2anyPV_flat',
'B0_FitPVConst_PV_Z_flat',
'B0_FitPVConst_X_flat',
'B0_FitPVConst_Y_flat',
'B0_FitPVConst_Z_flat',
'B0_FitPVConst_XERR_flat',
'B0_FitPVConst_YERR_flat',
'B0_FitPVConst_ZERR_flat',
#Jpsi
'B0_FitPVConst_J_psi_1S_MinIPCHI2anyPV_flat',
'B0_FitPVConst_J_psi_1S_IP_flat',
'B0_FitPVConst_J_psi_1S_IPCHI2_flat',
#KS0
'B0_FitPVConst_KS0_MinIPCHI2anyPV_flat',
'B0_FitPVConst_KS0_IP_flat',
'B0_FitPVConst_KS0_IPCHI2_flat',
'B0_FitPVConst_KS0_tau_flat',
'B0_FitPVConst_KS0_tauErr_flat',

# Other DTF stuff
'B0_FitJpsiPVConst_Z_flat',
'B0_FitJpsiConst_J_psi_1S_MinIPCHI2anyPV_flat',
'B0_FitSubstPVKpi1_KS0_MinIPCHI2anyPV',
#Random variables from Loki functors
'B0_BKGCAT',
'B0_TRUETAU',
'B0_TAU',
'B0_TAUERR',
'B0_MINIPCHI2',
'pv_z_pull',
'piplus_TRACK_Type',
'piminus_MINIPCHI2',
'piplus_MINIPCHI2',
'muminus_MINIPCHI2',
'muplus_MINIPCHI2',
'piplus_ProbNNp',
'piminus_ProbNNp',
'varLambda0MassHypo_ppluspiminus',
'varLambda0MassHypo_pminuspiplus',
'eventNumber',
'runNumber',
'B0_FitwithoutConst_chi2',
'B0_FitwithoutConst_nDOF',
'B0_LOKI_ETA',
'B0_LOKI_PHI',
'nPV',
'nTracks',
'idxPV',
'piplus_TRACK_Type',
'B0_TRUEID'
]

In [ ]:
#variables = 'B0_FitPVConst_IPCHI2'

In [9]:
sanity_cuts = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0'
trigger = '&(B0_L0MuonDecision_Dec==1|B0_L0DiMuonDecision_Dec==1|B0_L0MuonHighDecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt2DiMuonJPsiDecision_Dec==1|B0_Hlt2DiMuonDetachedJPsiDecision_Dec==1)'
cut = sanity_cuts+trigger
tree_data ='Bd2JpsiKs'

In [17]:
l0_cut = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0&(B0_L0MuonDecision_Dec==1|B0_L0DiMuonDecision_Dec==1|B0_L0MuonHighDecision_Dec==1)'
hlt1_cut = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)'
hlt2_cut = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0&(B0_Hlt2DiMuonJPsiDecision_Dec==1)'#|B0_Hlt2DiMuonDetachedJPsiDecision_Dec==1)'

In [18]:
if calculate_trigger_efficiency==True:
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns='B0_FitDaughtersConst_M_flat', flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns='B0_FitDaughtersConst_M_flat', flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_nocut = pd.concat([df_2015,df_2016])
    
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=sanity_cuts, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=sanity_cuts,flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_sanity = pd.concat([df_2015,df_2016])
    
    ##Check efficiencies of L0 stage
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=l0_cut, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=l0_cut,flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_l0= pd.concat([df_2015,df_2016])

    
    ##Check efficiencies of HLT1 stage
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=hlt1_cut, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=hlt1_cut,flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_hlt1 = pd.concat([df_2015,df_2016])
    
    ##Check efficiencies of HLT2 stage
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=hlt2_cut, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=hlt2_cut,flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_hlt2 = pd.concat([df_2015,df_2016])
    
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=cut, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=['B0_FitDaughtersConst_M_flat','piplus_TRACK_Type'], where=cut,flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc..")
    df_merged_trigger = pd.concat([df_2015,df_2016])
   
    
    efficiency_san = df_merged_sanity.shape[0] / df_merged_nocut.shape[0]
    efficiency_trigger = df_merged_trigger.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the sanity cuts is')
    print(efficiency_san)
    print('------------------------------------')
    print('the efficiency of the trigger requirements is:')
    print(efficiency_trigger)
    print('DD:')
    print(df_merged_trigger.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_trigger.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
   

    ##Print single trigger stages
    print('--------------------------------------')
    print("The efficiencies of the single trigger stages are:")
    print("L0")
    efficiency_l0 = df_merged_l0.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_l0)
    print('DD:')
    print(df_merged_l0.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_l0.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    print("------------------------------------")
    print("HLT1")
    efficiency_hlt1 = df_merged_hlt1.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_hlt1)
    print('DD:')
    print(df_merged_hlt1.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_hlt1.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    print("------------------------------------")
    print("HLT2")
    efficiency_hlt2 = df_merged_hlt2.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_hlt2)
    print('DD:')
    print(df_merged_hlt2.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_hlt2.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    
    
    print("------------------------------------")   
    
    efficiency_san = df_merged_sanity.shape[0] / df_merged_nocut.shape[0]
    efficiency_trigger = df_merged_trigger.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the sanity cuts is')
    print(efficiency_san)
    print('------------------------------------')
    print('the efficiency of the trigger requirements is:')
    print(efficiency_trigger)
    print('DD:')
    print(df_merged_trigger.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_trigger.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
   

    ##Print single trigger stages
    print('--------------------------------------')
    print("The efficiencies of the single trigger stages are:")
    print("L0")
    efficiency_l0 = df_merged_l0.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_l0)
    print('DD:')
    print(df_merged_l0.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_l0.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    print("------------------------------------")
    print("HLT1")
    efficiency_hlt1 = df_merged_hlt1.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_hlt1)
    print('DD:')
    print(df_merged_hlt1.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_hlt1.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    print("------------------------------------")
    print("HLT2")
    efficiency_hlt2 = df_merged_hlt2.shape[0] / df_merged_sanity.shape[0]
    print('the efficiency of the trigger requirements is:')
    print(efficiency_hlt2)
    print('DD:')
    print(df_merged_hlt2.query('piplus_TRACK_Type==5').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==5').shape[0])
    print('LL:')
    print(df_merged_hlt2.query('piplus_TRACK_Type==3').shape[0] / df_merged_sanity.query('piplus_TRACK_Type==3').shape[0])
    
    
    print("------------------------------------")
 

Reading 2015 mc.
The shape of 2015 mc is: (831240, 1)
Reading 2016 mc.
The shape of 2016 mc is: (828722, 1)
Merging 2015+2016 mc..
Reading 2015 mc.
The shape of 2015 mc is: (693869, 2)
Reading 2016 mc.
The shape of 2016 mc is: (692670, 2)
Merging 2015+2016 mc..
Reading 2015 mc.
The shape of 2015 mc is: (524367, 2)
Reading 2016 mc.
The shape of 2016 mc is: (578374, 2)
Merging 2015+2016 mc..
Reading 2015 mc.
The shape of 2015 mc is: (537903, 2)
Reading 2016 mc.
The shape of 2016 mc is: (562651, 2)
Merging 2015+2016 mc..
Reading 2015 mc.
The shape of 2015 mc is: (594891, 2)
Reading 2016 mc.
The shape of 2016 mc is: (599875, 2)
Merging 2015+2016 mc..
Reading 2015 mc.
The shape of 2015 mc is: (497704, 2)
Reading 2016 mc.
The shape of 2016 mc is: (546347, 2)
Merging 2015+2016 mc..
the efficiency of the sanity cuts is
0.8352835787807191
------------------------------------
the efficiency of the trigger requirements is:
0.7529907200590824
DD:
0.7492154156407738
LL:
0.7629577512857334
---------

In [19]:
create_tuple = True
if create_tuple==True:
    variables = [
    'B0_FitDaughtersConst_M_flat',
    'B0_FitDaughtersConst_chi2_flat',
    'B0_FitDaughtersConst_IPCHI2_flat',
    'B0_FitDaughtersConst_J_psi_1S_IP_flat',   
    'B0_FitDaughtersConst_KS0_P1_PT_flat',
    'B0_FitDaughtersConst_KS0_P0_PT_flat',  
    'B0_FitDaughtersConst_KS0_decayLength_flat',
    'B0_FitDaughtersConst_KS0_IP_flat',
    'B0_FitDaughtersConst_KS0_P0_IPCHI2_flat', 
    'B0_FitDaughtersConst_J_psi_1S_IPCHI2_flat',
    'idxPV',
    'B0_FitDaughtersConst_KS0_P1_PT_flat', 
    'B0_FitDaughtersConst_KS0_P0_PT_flat',
    'B0_FitDaughtersConst_KS0_P0_IPCHI2_flat', 
    'B0_FitDaughtersConst_KS0_P1_IPCHI2_flat',
    'B0_FitDaughtersConst_J_psi_1S_P0_PT_flat', 
    'B0_FitDaughtersConst_J_psi_1S_P1_PT_flat',
    'eventNumber',
    'runNumber',
    'nPV',
    'nPV',
    'nTracks',  
    ]
    #directories and files for 2015
    data_file_2015 = 'Bd2JpsimumuKS_mc_2015_flat.root'
    data_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/'
    data_2015 = data_dir_2015 + data_file_2015
    #read from ROOT-fil
    print("Reading 2015 data.")
    df_2015 = rp.read_root(data_2015,key=tree_data, columns=variables, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    #directories and files for 2015
    data_file_2016 = 'Bd2JpsimumuKS_mc_2016_flat.root'
    data_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/'
    data_2016 = data_dir_2016 + data_file_2016
    #read from ROOT-file
    print("Reading 2016 data.")
    df_2016 = rp.read_root(data_2016,key=tree_data, columns=variables, flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("Merging 2015+2016 data and write to ROOT file.")
    df_merged = pd.concat([df_2015,df_2016])
    #Calculate random variable for random selection
    np.random.seed(42)
    df_merged['idxRandom'] = np.random.choice(2**30, df_merged.shape[0])  
    df_merged['idxEventNumber'] = df_merged['eventNumber'] 
    df_merged['idxRunNumber'] = df_merged['runNumber'] 
    df_merged.to_root(mc_dir_2015_2016+'Bd2JpsiKS.root',key='Bd2JpsiKS')

Reading 2015 data.


KeyboardInterrupt: 

In [10]:
if create_sanity_tuple==True:
    #directories and files for 2015
    mc_file_2015 = 'Bd2JpsimumuKS_mc_2015.root'
    mc_dir_2015 = '/fhgfs/users/chasenberg/mc/2015/jpsimumuks/work_cut/veto_leaves/wrong_pv_check/'
    mc_2015 = mc_dir_2015 + mc_file_2015
    #read from ROOT-fil
    print("Reading 2015 mc.")
    df_2015 = rp.read_root(mc_2015,key=tree_data, columns=variables, flatten=False)
    df_2015 = df_2015.replace([np.inf, -np.inf], np.nan)
    df_2015 = df_2015.dropna()
    print("The shape of 2015 mc is:",df_2015.shape)
    #directories and files for 2015
    mc_file_2016 = 'Bd2JpsimumuKS_mc_2016.root'
    mc_dir_2016 = '/fhgfs/users/chasenberg/mc/2016/jpsimumuks/work_cut/veto_leaves/wrong_pv/'
    mc_2016 = mc_dir_2016 + mc_file_2016
    #read from ROOT-file
    print("Reading 2016 mc.")
    df_2016 = rp.read_root(mc_2016,key=tree_data, columns=variables, flatten=False)
    df_2016 = df_2016.replace([np.inf, -np.inf], np.nan)
    df_2016 = df_2016.dropna()
    print("The shape of 2016 mc is:",df_2016.shape)
    print("Merging 2015+2016 mc and write to ROOT file.")
    df_merged = pd.concat([df_2015,df_2016])
    mc_dir_2015_2016 = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/'
    file_name = 'info.txt'
    file = open(mc_dir_2015_2016+file_name,'w')  
    file.write("----------------------------------------")
    file.write("The ROOT file has got the calculated L0 veto mass and the pv_z_pull variable, to discriminate between correct/incorrect pvs.")
    file.write('The file'+mc_sanity_cuts+'has got the following cuts:\n') 
    file.write(sanity_cuts)
    file.write('No other selection has been applied yet.\n')
    size = str(df_merged.shape[0])
    file.write('The file has'+size+'entries\n')
    file.write('----------------------------------------')
    #Calculate random variable for random selection
    np.random.seed(42)
    df_merged['idxRandom'] = np.random.choice(2**30, df_merged.shape[0])  
    df_merged['idxEventNumber'] = df_merged['eventNumber'] 
    df_merged['idxRunNumber'] = df_merged['runNumber'] 
    df_merged.to_root(mc_dir_2015_2016+mc_sanity_cuts,key='Bd2JpsiKS')

Reading 2015 mc.
The shape of 2015 mc is: (493853, 70)
Reading 2016 mc.
The shape of 2016 mc is: (543112, 70)
Merging 2015+2016 mc and write to ROOT file.


/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/root_numpy/_tree.py:570: UserWarning: converter for dtype('O') is not implemented (skipping)
  _librootnumpy.array2root(arr, filename, treename, mode)


## Efficiency part

### Read selected data: BDT-cut is 1.15 according to FOM

In [11]:
mc_selected = 'Bd2JpsiKS_selected.root'
mc_selected = mc_dir_2015_2016 + mc_selected
df_selected = rp.read_root(mc_selected)
#Calculate true time error 
#signal_dataframe_wrongPV['Delta_TAU'] = signal_dataframe_wrongPV['B0_FitPVConst_tau_flat']-signal_dataframe_wrongPV['B0_TRUETAU']

In [24]:
df_selected.columns

Index(['B0_TAGOMEGA_OS', 'B0_TAGDECISION_OS',
       'B0_FitDaughtersConst_status_flat', 'B0_FitDaughtersConst_M_flat',
       'B0_FitDaughtersConst_chi2_flat', 'B0_FitDaughtersConst_IPCHI2_flat',
       'B0_FitDaughtersConst_nDOF_flat', 'B0_FitDaughtersConst_PT_flat',
       'B0_FitDaughtersConst_J_psi_1S_P0_PT_flat',
       'B0_FitDaughtersConst_J_psi_1S_P1_PT_flat',
       'B0_FitDaughtersConst_J_psi_1S_MinIPCHI2anyPV_flat',
       'B0_FitDaughtersConst_J_psi_1S_IPCHI2_flat',
       'B0_FitDaughtersConst_J_psi_1S_IP_flat',
       'B0_FitDaughtersConst_KS0_P1_PT_flat',
       'B0_FitDaughtersConst_KS0_P0_PT_flat',
       'B0_FitDaughtersConst_KS0_P0_IPCHI2_flat',
       'B0_FitDaughtersConst_KS0_P1_IPCHI2_flat',
       'B0_FitDaughtersConst_KS0_decayLength_flat',
       'B0_FitDaughtersConst_KS0_IP_flat', 'B0_FitPVConst_status_flat',
       'B0_FitPVConst_IPCHI2_flat', 'B0_FitPVConst_chi2_flat',
       'B0_FitPVConst_nDOF_flat', 'B0_FitPVConst_tauErr_flat',
       'B0_FitPVConst_tau_

In [26]:
mc_pv_selected = 'Bd2JpsiKS_pv_selected.root'
mc_pv_selected = mc_dir_2015_2016 + mc_pv_selected
df_pv_selected = rp.read_root(mc_pv_selected)
#Calculate true time error 
#signal_dataframe_wrongPV['Delta_TAU'] = signal_dataframe_wrongPV['B0_FitPVConst_tau_flat']-signal_dataframe_wrongPV['B0_TRUETAU']pv_

### Best PV cut

In [ ]:
#Check best PV cut
efficiency = df_selected.query('idxPV==0').shape[0]/df_selected.shape[0]
print("The efficency of the best pv selection is:")
print(efficiency)
print("----------------------------------------")

### Write a best PV selected dataset to root

In [28]:
l0_cut =  '((abs(1115.683-varLambda0MassHypo_ppluspiminus)>17)|piplus_ProbNNp<0.4)&((abs(1115.683-varLambda0MassHypo_pminuspiplus)>17)|(piminus_ProbNNp<0.4))'
pv_cut = "&idxPV==0"# "&B0_FitPVConst_MinIPCHI2anyPV_flat>6"
df_pv_selected.query(l0_cut + pv_cut).to_root(mc_dir_2015_2016+'Bd2JpsiKS_bestPV.root', key='Bd2JpsiKS')

Check efficiencies of all selection steps
=================================================

BDT cut of 1.15
------------------------

In [ ]:
efficiency =  df_selected.query('idxPV==0').shape[0]/df_merged.query('idxPV==0').shape[0]
print("The efficency of the BDT cut is:")
print(efficiency)
print("----------------------------------------")

Apply the L0 veto
--------------------------

In [ ]:
l0_cut =  '((abs(1115.683-varLambda0MassHypo_ppluspiminus)>17)|piplus_ProbNNp<0.4)&((abs(1115.683-varLambda0MassHypo_pminuspiplus)>17)|(piminus_ProbNNp<0.4))'

In [ ]:
l0_cutefficiency = df_selected.query(l0_cut).query('idxPV==0').shape[0]/df_selected.query('idxPV==0').shape[0]
print("The efficency of the L0 veto is:")
print(l0_cutefficiency)
print("----------------------------------------")

Apply the MinIPCHI2 cut > 6 and perform a random candidate selection
--------------------------

### Write new ROOT File with L0 veto and MinIPCHI2anyPV cut

In [13]:
l0_cut =  '((abs(1115.683-varLambda0MassHypo_ppluspiminus)>17)|piplus_ProbNNp<0.4)&((abs(1115.683-varLambda0MassHypo_pminuspiplus)>17)|(piminus_ProbNNp<0.4))'
pv_cut = ""# "&B0_FitPVConst_MinIPCHI2anyPV_flat>6"
df_selected.query(l0_cut + pv_cut).to_root(mc_dir_2015_2016+'Bd2JpsiKS_cuts.root', key='Bd2JpsiKS')

In [29]:
import os, subprocess
if create_randomSel_tuple==True:
    print('INFO: Call CandidateSelectionGrimReaper', flush=True)
    my_env = os.environ.copy()
    my_env['PATH'] = '/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/chasenberg/bin'
    my_env['LD_LIBRARY_PATH'] = '/usr/local/lib'
    my_env['script'] = '/home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection'
    print("Start CandidateSelectionGrimReaper!")
    my_command = 'source /lhcbsoft/LHCbSoftwareSetup.sh &&' \
             'lb-run DaVinci/v41r2 $BASH -c "' \
             'echo $PATH && ' \
             'source /doosoft/InstallDooSoftware/LoadDooSoftware && CandidateSelectionGrimReaper /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/Bd2JpsiKS_pv_selected.root Bd2JpsiKS /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/Bd2JpsiKS_random.root Bd2JpsiKS "idxRandom""'                     
    subprocess.Popen([my_command], env=my_env, shell=True)
    print("Wait for GrimReaper")
    time.sleep(120.0)    # pause 5.5 seconds
    print("GrimReaper hopefully finished")

INFO: Call CandidateSelectionGrimReaper
Start CandidateSelectionGrimReaper!
Wait for GrimReaper
GrimReaper hopefully finished


Efficiency of random selection
-------------------------------

In [ ]:
df_random_sel = rp.read_root(mc_dir_2015_2016 + mc_random, key='Bd2JpsiKS')

In [ ]:
eff = df_random_sel.shape[0]/df_selected.query(l0_cut+'&idxPV==0').shape[0]
print(eff)